# Model 1: M/M/c
Based on Lab 5 code

In [1]:
"""(q4.py) M/M/c queueing system with several monitors and multiple replications"""

from SimPy.Simulation import *
import random
import numpy
import math

In [2]:
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return lower, upper

In [3]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(mu))
            t = random.expovariate(lamb)
            yield hold, self, t

In [4]:
class Arrival(Process):
    """an arrival"""
    n = 0  # class variable (number in system)

    def run(self, mu):
        # Event: arrival
        Arrival.n += 1   # number in system
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)

        yield request, self, G.server
        # ... waiting in queue for server to be empty (delay) ...
      
        # Event: service begins
        t = random.expovariate(mu)
      
        yield hold, self, t
        # ... now being served (activity) ...
      
        # Event: service ends
        yield release, self, G.server # let go of server (takes no simulation time)
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

In [5]:
class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'
    busymon = 'Monitor'

In [6]:
def model(c, N, lamb, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()
    
    Arrival.n = 0
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, mu))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    return W, L, B

In [7]:
## Experiment ----------------
allW = []
allL = []
allB = []
allLambdaEffective = []
for k in range(50):
    seed = 123*k
    result = model(c=2, N=10000, lamb=1/2.445357142857143, mu=1/3.928647959183673, maxtime=200000, rvseed=seed)
    allW.append(result[0])
    allL.append(result[1])
    allB.append(result[2])
    allLambdaEffective.append(result[1]/result[0])

In [8]:
print("Estimate of W:", numpy.mean(allW))
print("Conf int of W:", conf(allW))
print("Estimate of L:", numpy.mean(allL))
print("Conf int of L:", conf(allL))
print("Estimate of B:", numpy.mean(allB))
print("Conf int of B:", conf(allB))
print("Estimate of LambdaEffective:", numpy.mean(allLambdaEffective))
print("Conf int of LambdaEffective:", conf(allLambdaEffective))

Estimate of W: 11.095118080820878
Conf int of W: (10.8503271254504, 11.339909036191356)
Estimate of L: 4.5396422644130965
Conf int of L: (4.433052130038653, 4.64623239878754)
Estimate of B: 0.8903000548365461
Conf int of B: (0.8884404255669028, 0.8921596841061893)
Estimate of LambdaEffective: 0.4090164546568189
Conf int of LambdaEffective: (0.4077578201132859, 0.41027508920035194)
